In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/Final_Project

/content/drive/MyDrive/Final_Project


In [ ]:
cd DATN/

/content/drive/MyDrive/Final_Project/DATN


In [ ]:
pip install fuzzywuzzy

In [ ]:
import numpy,datetime,re
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from difflib import SequenceMatcher

/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
tentative_engagements = []
free_bds = []
corresponacy1 = []

In [ ]:
def matchingtablebyheaders(tbl1,tbl2):
    # For matching
    Pref=np.zeros((len(tbl1),len(tbl2)))
        #assign the matching ratio between tbl1 and tbl2
    for i in range(len(tbl1)):
        for j in range(len(tbl2)):
            Pref[i,j] = np.int(100*SequenceMatcher(None,tbl1[i].lower(),tbl2[j].lower()).ratio())
    return Pref

In [ ]:
def listofpreferencespertable(tbl1,tbl2,Pref):
        #for chotot
    ipref_tom=np.zeros((len(tbl1),len(tbl2)),dtype=np.int8)
    #for bds
    ipref_bds=np.zeros((len(tbl2),len(tbl1)),dtype=np.int8)
    pref_bds=np.zeros((len(tbl2),len(tbl1)),dtype=np.int8)
    preferred_rankings_data_1 = {}
    preferred_rankings_data_2 = {}
    #create list of preferred_rankings_chotot with header
    #assign the array indexs of Pref sorted by the actual values decending based on the lenght of tbl2 (17)
    for i in range(len(tbl2)):
        ipref_tom[:,i]=np.argsort(Pref[:,i])
        ipref_tom[:,i]=ipref_tom[::-1,i]
        preferred_rankings_data_1[i] = ipref_tom[:,i]
        #pref_tom[:,i]=np.sort(Pref[:,i])
        #pref_tom[:,i]=pref_tom[::-1,i]
    #ipref_tom

    #create list of preferred_rankings_bds with header
    #assign the array indexs of Pref sorted by the actual values decending based on the lenght of tbl1 (13)
    for i in range(len(tbl1)):
        #get indexs values of rows and store them in the rows as well
        ipref_bds[:,i] = np.argsort(Pref[i,:])
        ipref_bds[:,i] = ipref_bds[::-1,i]
        preferred_rankings_data_2[i] = ipref_bds[:,i]
        #pref_bds[:,i] = np.sort(Pref[i,:])
      #  pref_bds[:,i] = pref_bds[::-1,i]
    #ipref_bds
    return preferred_rankings_data_2,preferred_rankings_data_1

In [ ]:
#Imdb which still need to propose and get accepted successfully
#Keep track of the headers that "may" end up together
def init_free_bds(preferred_rankings_imdb):
    for imdb in preferred_rankings_imdb.keys():
        free_bds.append(imdb)

In [ ]:
def begin_matching(bds,preferred_rankings_data_1,preferred_rankings_data_2):
    #  '''Find the first free Tomaten available to a bds at any given time'''
    #print(preferred_rankings_data_1[bds])
    for tomate in preferred_rankings_data_1[bds]:
        #Boolean for whether tomate is taken or not
        taken_match = [couple for couple in tentative_engagements if tomate == couple [:][1]]
        if (len(taken_match) == 0):
            #tentatively engage the man and woman
            tentative_engagements.append([bds, tomate])
            free_bds.remove(bds)
            break
        elif (len(taken_match) > 0):

            current_bds = list(preferred_rankings_data_2[tomate].tolist()).index(taken_match[0][0])
            potential_bds = list(preferred_rankings_data_2[tomate].tolist()).index(bds)

            if (current_bds < potential_bds):
                ii=[]

            else:
                free_bds.remove(bds)
                #The old bds is now single
                free_bds.append(taken_match[0][0])
                #Update the fiance of the tomaten (tentatively)
                taken_match[0][0] = bds
                break

In [ ]:
def stable_matching(preferred_rankings_data_1,preferred_rankings_data_2):
    '''Matching algorithm until stable match terminates'''
    while (len(free_bds) > 0):
        # print((free_bds))
        for imdb in free_bds:

            begin_matching(imdb,preferred_rankings_data_1,preferred_rankings_data_2)
        # print((free_bds))
        # break

In [ ]:
def check_short_matching(COL1,COL2):
    rate = 0.0
    if str(COL1) == 'nan' or str(COL2) == 'nan' :
        return 0
    COL1 = str(COL1)
    COL2 = str(COL2)
    sw = (fuzz.SequenceMatcher(None,COL1, COL2).ratio())
    return sw #/(max(len(COL1),len(COL2))*1.0)


In [ ]:
def check_match(st1,st2):
    str1 = np.array((st1.values).astype(str))
    str2 = np.array((st2.values).astype(str))
    # ret = np.zeros((len(str1),len(str2)))
    ret = np.zeros((len(str1)))

    for i in range(len(str1)):
        tmp = -1
        for j in range(len(str2)):
            # ret[i,j] = check_short_matching(str1[i],str2[j])
            tmp = max(tmp, check_short_matching(str1[i],str2[j]))
        ret[i] = tmp
    return numpy.average(ret)

In [ ]:
def matchingtablebyvalues(DATA_1_DF_SAMPLE,DATA_2_DF_SAMPLE,tbl1,tbl2):
    dd = []
    ret = []
    value1 = []
    value2 = []
    index_i = 0
    Pref=np.zeros((len(tbl1),len(tbl2)))
    for i in DATA_1_DF_SAMPLE:
        dd = []
        index_j = 0
        for j in DATA_2_DF_SAMPLE:
            table1 = DATA_1_DF_SAMPLE[i]
            table2 = DATA_2_DF_SAMPLE[j]
            value1 = (table1.values).astype(str)
            value2 = (table2.values).astype(str)
            #dd.append([ 100* (check_match(table1, table2)),i,j])
            Pref[index_i,index_j] = 100 * (np.float(check_match(table1, table2)))
            index_j+=1
        index_i+=1
    return Pref

In [ ]:
def outputforvalues(tbl1,tbl2,Pref):
    i = 0
    for item in tentative_engagements:
        matching=Pref[item[0],item[1]]
        print(tbl1[item[0]] + " -----" ,tbl2[item[1]],':',matching)

# batdongsan123 vs alonhadat

In [ ]:
alonhadat = pd.read_csv("/content/drive/MyDrive/newcode/alonhadat.csv", index_col=0)

In [ ]:
alonhadat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 828 entries, 0 to 1589
Data columns (total 28 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   bedroom            828 non-null    object 
 1   code               827 non-null    float64
 2   date               827 non-null    object 
 3   description        828 non-null    object 
 4   diningroom         828 non-null    object 
 5   direct             827 non-null    object 
 6   floor              828 non-null    object 
 7   introduce_contact  828 non-null    object 
 8   juridical          827 non-null    object 
 9   kitchen            828 non-null    object 
 10  length             827 non-null    object 
 11  link_image         828 non-null    object 
 12  name_contact       827 non-null    object 
 13  parking            828 non-null    object 
 14  phone_contact      827 non-null    object 
 15  price              828 non-null    int64  
 16  project            828 no

In [ ]:
alonhadat = pd.DataFrame(alonhadat[['title', 'description', 'price', 'square', 'type', 'name_contact', 'phone_contact', 'date', 'direct', 'district', 'province', 'street', 'ward', 'floor', 'juridical', 'bedroom', 'length', 'width', 'link_image', 'url_page', 'kitchen', 'parking', 'terrace']])

In [ ]:
batdongsan123 = pd.read_csv("/content/drive/MyDrive/newcode/bds123.csv", index_col=0)

In [ ]:
batdongsan123['length'] = '0 m'
batdongsan123['width'] = '0 m'

In [ ]:
batdongsan123.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9648 entries, 0 to 9998
Data columns (total 26 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   acreage        9648 non-null   object 
 1   bathroom       9648 non-null   object 
 2   bedroom        9648 non-null   object 
 3   code           9648 non-null   int64  
 4   date           9648 non-null   object 
 5   description    9648 non-null   object 
 6   direction      9648 non-null   object 
 7   link_image     8798 non-null   object 
 8   name_contact   9648 non-null   object 
 9   phone_contact  9648 non-null   int64  
 10  price          9648 non-null   float64
 11  project        9648 non-null   object 
 12  title          9648 non-null   object 
 13  type           9648 non-null   int64  
 14  url_page       9648 non-null   object 
 15  ward           9648 non-null   object 
 16  district       9648 non-null   object 
 17  province       9648 non-null   object 
 18  street  

In [ ]:
def main():
    tentative_engagements.clear()
    free_bds.clear()
    #add data
    # DATA_SAMPLE_1,tbl1,DATA_SAMPLE_2,tbl2=importdatasets()
    DATA_SAMPLE_1 = alonhadat.sample(50).astype(str)

    header_1 = list(alonhadat.columns.values)
    tbl1 = np.array(header_1)

    DATA_SAMPLE_2 = alonhadat.sample(50).astype(str)

    header_2 = list(alonhadat.columns.values)
    tbl2 = np.array(header_2)

    #creation templates
    Pref_header=matchingtablebyheaders(tbl1,tbl2)
    Pref_values=matchingtablebyvalues(DATA_SAMPLE_1,DATA_SAMPLE_2,tbl1,tbl2)
    Pref = (0.5*Pref_header + 0.5*Pref_values)
    preferred_rankings_imdb,preferred_rankings_tomatoes=listofpreferencespertable(tbl1,tbl2,Pref)
    # print(preferred_rankings_tomatoes)
    init_free_bds(preferred_rankings_imdb)
    stable_matching(preferred_rankings_imdb,preferred_rankings_tomatoes)
    # print(tentative_engagements)
    outputforvalues(tbl1,tbl2,Pref)



main()

<ipython-input-7-4e98165c491e>:7: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  Pref[i,j] = np.int(100*SequenceMatcher(None,tbl1[i].lower(),tbl2[j].lower()).ratio())
<ipython-input-14-c247a0c48378>:17: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-

title ----- title : 73.40098875957368
price ----- price : 97.04415534884643
type ----- type : 100.0
phone_contact ----- phone_contact : 85.67435897435897
direct ----- direct : 100.0
province ----- province : 97.38571428571429
ward ----- ward : 81.1499716142131
juridical ----- juridical : 99.21428571428572
length ----- length : 97.95714285714286
link_image ----- link_image : 92.90684597760175
kitchen ----- kitchen : 100.0
terrace ----- terrace : 100.0
description ----- description : 64.48130281388852
name_contact ----- name_contact : 83.95794920753687
district ----- district : 87.70872051924684
floor ----- floor : 99.66666666666666
width ----- width : 96.61825396825397
parking ----- parking : 100.0
square ----- square : 95.28138528138528
street ----- street : 81.00999618564073
url_page ----- url_page : 84.72223632924195
date ----- date : 99.8
bedroom ----- bedroom : 98.94444444444444


In [ ]:
batdongsan123 = batdongsan123.rename(columns={'acreage': 'square'})
batdongsan123 = batdongsan123.rename(columns={'direction': 'direct'})

In [ ]:
batdongsan123 = pd.DataFrame(batdongsan123[['title', 'description', 'price', 'square', 'type', 'name_contact', 'phone_contact', 'date', 'direct', 'district', 'province', 'street', 'ward', 'floor', 'juridical', 'bedroom', 'length', 'width', 'link_image', 'url_page', 'kitchen', 'parking', 'terrace']])

# alonhadat vs ibatdongsan

In [ ]:
ibatdongsan = pd.read_csv("/content/drive/MyDrive/newcode/ibds.csv", index_col=0)

In [ ]:
ibatdongsan.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8628 entries, 0 to 8883
Data columns (total 26 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   area           8628 non-null   object
 1   bedroom        8628 non-null   object
 2   brief          8628 non-null   object
 3   code           8628 non-null   int64 
 4   date           8628 non-null   object
 5   diningroom     8628 non-null   object
 6   direct         8628 non-null   object
 7   floor          8628 non-null   object
 8   juridical      8628 non-null   object
 9   kitchen        8628 non-null   object
 10  length         8628 non-null   object
 11  link_image     8628 non-null   object
 12  name_contact   8628 non-null   object
 13  parking        8628 non-null   object
 14  phone_contact  8628 non-null   object
 15  price          8628 non-null   int64 
 16  terrace        8628 non-null   object
 17  title          8628 non-null   object
 18  type           8628 non-null

In [ ]:
def main():
    tentative_engagements.clear()
    free_bds.clear()
    #add data
    # DATA_SAMPLE_1,tbl1,DATA_SAMPLE_2,tbl2=importdatasets()
    DATA_SAMPLE_1 = alonhadat.sample(50).astype(str)

    header_1 = list(alonhadat.columns.values)
    tbl1 = np.array(header_1)

    DATA_SAMPLE_2 = ibatdongsan.sample(50).astype(str)

    header_2 = list(ibatdongsan.columns.values)
    tbl2 = np.array(header_2)

    #creation templates
    Pref_header=matchingtablebyheaders(tbl1,tbl2)
    Pref_values=matchingtablebyvalues(DATA_SAMPLE_1,DATA_SAMPLE_2,tbl1,tbl2)
    Pref = (0.5*Pref_header + 0.5*Pref_values)
    preferred_rankings_imdb,preferred_rankings_tomatoes=listofpreferencespertable(tbl1,tbl2,Pref)
    # print(preferred_rankings_tomatoes)
    init_free_bds(preferred_rankings_imdb)
    stable_matching(preferred_rankings_imdb,preferred_rankings_tomatoes)
    # print(tentative_engagements)
    outputforvalues(tbl1,tbl2,Pref)



main()

<ipython-input-7-4e98165c491e>:7: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  Pref[i,j] = np.int(100*SequenceMatcher(None,tbl1[i].lower(),tbl2[j].lower()).ratio())
<ipython-input-14-c247a0c48378>:17: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-

title ----- title : 70.21495646786025
price ----- price : 96.11682731202183
type ----- type : 100.0
phone_contact ----- phone_contact : 82.15212121212122
direct ----- direct : 97.71666666666667
province ----- province : 86.48123607558591
ward ----- ward : 77.77162670812085
juridical ----- juridical : 96.6842105263158
length ----- length : 95.3177489177489
link_image ----- link_image : 79.44793040402696
kitchen ----- kitchen : 100.0
terrace ----- terrace : 100.0
district ----- district : 91.47734133771286
name_contact ----- name_contact : 76.9938289911732
floor ----- floor : 100.0
width ----- width : 94.2015873015873
parking ----- parking : 100.0
square ----- area : 62.77519702519703
street ----- street : 79.49608582338828
url_page ----- url_page : 77.81550846272168
date ----- date : 94.5
description ----- brief : 20.11709696196634
bedroom ----- bedroom : 99.52777777777777


In [ ]:
ibatdongsan = ibatdongsan.rename(columns={'area': 'square'})
ibatdongsan = ibatdongsan.rename(columns={'brief': 'description'})

In [ ]:
ibatdongsan = pd.DataFrame(ibatdongsan[['title', 'description', 'price', 'square', 'type', 'name_contact', 'phone_contact', 'date', 'direct', 'district', 'province', 'street', 'ward', 'floor', 'juridical', 'bedroom', 'length', 'width', 'link_image', 'url_page', 'kitchen', 'parking', 'terrace']])

# alonhadat vs nhadat24h

In [ ]:
nhadat24h = pd.read_csv("/content/drive/MyDrive/newcode/nhadat24h.csv", index_col=0)

In [ ]:
nhadat24h.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1540 entries, 0 to 1963
Data columns (total 28 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   bathroom          1540 non-null   object 
 1   bedroom           1540 non-null   object 
 2   code              1540 non-null   object 
 3   date              0 non-null      float64
 4   description       1540 non-null   object 
 5   direct            244 non-null    object 
 6   floor             1540 non-null   object 
 7   ground_area       1540 non-null   object 
 8   juridical         1540 non-null   object 
 9   kitchen           1540 non-null   object 
 10  length            1540 non-null   object 
 11  link_image        1540 non-null   object 
 12  livingroom        1540 non-null   object 
 13  name_project      1540 non-null   object 
 14  parking           1540 non-null   object 
 15  price             1540 non-null   int64  
 16  road_width        1540 non-null   object 


In [ ]:
nhadat24h['name_contact'] = np.nan
nhadat24h['phone_contact'] = np.nan

In [ ]:
def main():
    tentative_engagements.clear()
    free_bds.clear()
    #add data
    # DATA_SAMPLE_1,tbl1,DATA_SAMPLE_2,tbl2=importdatasets()
    DATA_SAMPLE_1 = alonhadat.sample(50).astype(str)

    header_1 = list(alonhadat.columns.values)
    tbl1 = np.array(header_1)

    DATA_SAMPLE_2 = nhadat24h.sample(50).astype(str)

    header_2 = list(nhadat24h.columns.values)
    tbl2 = np.array(header_2)

    #creation templates
    Pref_header=matchingtablebyheaders(tbl1,tbl2)
    Pref_values=matchingtablebyvalues(DATA_SAMPLE_1,DATA_SAMPLE_2,tbl1,tbl2)
    Pref = (0.5*Pref_header + 0.5*Pref_values)
    preferred_rankings_imdb,preferred_rankings_tomatoes=listofpreferencespertable(tbl1,tbl2,Pref)
    # print(preferred_rankings_tomatoes)
    init_free_bds(preferred_rankings_imdb)
    stable_matching(preferred_rankings_imdb,preferred_rankings_tomatoes)
    # print(tentative_engagements)
    outputforvalues(tbl1,tbl2,Pref)



main()

<ipython-input-7-4e98165c491e>:7: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  Pref[i,j] = np.int(100*SequenceMatcher(None,tbl1[i].lower(),tbl2[j].lower()).ratio())
<ipython-input-14-c247a0c48378>:17: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-

title ----- title : 69.13047706909511
price ----- price : 95.7223866649496
type ----- type : 100.0
phone_contact ----- phone_contact : 50.0
district ----- district : 92.24360098965361
province ----- province : 86.2993932015099
ward ----- ward : 79.12534601621289
juridical ----- juridical : 80.85714285714286
length ----- length : 82.85656565656566
link_image ----- link_image : 74.65020899678589
kitchen ----- kitchen : 93.16363636363636
terrace ----- terrace : 93.38181818181818
description ----- description : 59.30295927241597
name_contact ----- name_contact : 50.0
floor ----- floor : 92.38095238095238
width ----- width : 81.61053391053392
parking ----- parking : 93.6
square ----- ground_area : 68.64671994671994
street ----- street : 74.9685854756431
url_page ----- url_page : 75.8945956214055
date ----- code : 51.0
direct ----- direct : 73.99545454545455
bedroom ----- bedroom : 97.77777777777777


In [ ]:
nhadat24h = nhadat24h.rename(columns={'ground_area': 'square'})

In [ ]:
nhadat24h = pd.DataFrame(nhadat24h[['title', 'description', 'price', 'square', 'type', 'name_contact', 'phone_contact', 'date', 'direct', 'district', 'province', 'street', 'ward', 'floor', 'juridical', 'bedroom', 'length', 'width', 'link_image', 'url_page', 'kitchen', 'parking', 'terrace']])

# Concat data to one table

In [ ]:
dataset = pd.concat([alonhadat, batdongsan123, ibatdongsan, nhadat24h])

In [ ]:
dataset = dataset.reset_index(drop=True)

In [ ]:
dataset

,title,description,price,square,type,name_contact,phone_contact,date,direct,district,...,floor,juridical,bedroom,length,width,link_image,url_page,kitchen,parking,terrace
0,Bán Gấp Nhà 2 Tầng 2 Mê Kiên cố MT Đường 10.5m...,Chủ CẦN Tiền Bán Gấp Nhà 2 Tầng Đẹp Kiên Cố Gi...,6.350000e+09,140 m,0,Văn Huy,0903.533.822,26/11/2023,Tây Bắc,Liên Chiểu,...,2 t,Sổ hồng/ Sổ đỏ,5 pn,28m,5m,https://alonhadat.com.vn/files/properties/2023...,https://alonhadat.com.vn/ban-gap-nha-2-tang-2-...,Có,Có,Có
1,"Nhà hẻm xe hơi Thuận Kiều quận 5, 48m2 2Tỷ850,...",Vị trí: Hẻm đường Thuận Kiều phường 12 quận 5\...,2.850000e+09,48 m,0,Trần Vũ Duy,0765.062.588,26/11/2023,None,5,...,1 t,Sổ hồng/ Sổ đỏ,2 pn,"12,6m","3,8m",https://alonhadat.com.vn/files/properties/2023...,https://alonhadat.com.vn/nha-hem-xe-hoi-thuan-...,Có,Có,Có
2,"Nhà sổ riêng Võ Trường Toản phường 15 quận 5, ...",Vị trí: hẻm đường Võ Trường Toản phường 15 quậ...,2.400000e+09,42 m,0,Trần Vũ Duy,0765.062.588,26/11/2023,None,5,...,1 t,Sổ hồng/ Sổ đỏ,2 pn,"10,5m",4m,https://alonhadat.com.vn/files/properties/2023...,https://alonhadat.com.vn/nha-so-rieng-vo-truon...,Có,Có,Có
3,"GIÁ TỐT ĐÂY Nhà 1/ Nguyễn Thượng Hiền, P.5, Qu...","GIÁ TỐT ĐÂY Nhà 1/ Nguyễn Thượng Hiền, P.5, Qu...",2.820000e+09,52 m,0,Hải Đăng,0903.315.821,26/11/2023,None,3,...,2 t,Sổ hồng/ Sổ đỏ,3 pn,13m,4m,https://alonhadat.com.vn/files/properties/2023...,https://alonhadat.com.vn/gia-tot-day-nha-1-ngu...,Không,Không,Không
4,HẺM XE TẢI TRÁNH - 79M2 - NGANG 6M - 3 TẦNG - ...,💥💥💥SIÊU PHẨM GIẢM GIÁ 🔥🔥🔥\n\n🚛🚛 HẺM XE TẢI TRÁ...,8.480000e+09,79 m,0,Nguyễn Tuấn,0833.677.555,26/11/2023,None,Gò Vấp,...,3 t,Sổ hồng/ Sổ đỏ,4 pn,13m,6m,https://alonhadat.com.vn/files/properties/2023...,https://alonhadat.com.vn/hem-xe-tai-tranh-79m2...,Có,Có,Có
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20639,Chính chủ cần bán lô đất tại Xã Kỳ Phú-Huyện N...,None,3.000000e+09,371 m,0,NaN,NaN,NaN,NaN,Nho Quan,...,0,Sổ Đỏ,0,0 m,0 m,https://nhadat24h.net.vn/storage/product/5466/...,https://nhadat24h.net.vn/du-an/chinh-chu-can-b...,Không,Không,Không
20640,Bán Nhà Mặt Phố Đê La Thành Quận Đống Đa. 77m ...,"30M RA NGÃ 5 Ô CHỢ DỪA, NHÀ MẶT PHỐ, GIÁ TRONG...",1.250000e+10,77 m,0,NaN,NaN,NaN,NaN,Đống Đa,...,2 t,Sổ Đỏ,2 pn,19.25 m,4 m,https://nhadat24h.net.vn/storage/product/11106...,https://nhadat24h.net.vn/du-an/ban-nha-mat-pho...,Có,Không,Không
20641,HẺM XE HƠI VÔ NHÀ - 44M2 - 5 TẦNG - AN NINH - ...,None,6.500000e+09,44 m,0,NaN,NaN,NaN,NaN,Tân Bình,...,5 t,Sổ Hồng,5 pn,11 m,4 m,https://nhadat24h.net.vn/storage/product/5568/...,https://nhadat24h.net.vn/du-an/hem-xe-hoi-vo-n...,Có,Không,Không
20642,Bán Nhà Phố Thụy Khuê Quận Tây Hồ. 88m Xây 7 T...,"Sổ cất két, sẵn giao dịch ngay.",1.260000e+10,88 m,0,NaN,NaN,NaN,NaN,Tây Hồ,...,12 t,Sổ Đỏ,7 pn,22 m,4 m,https://nhadat24h.net.vn/storage/product/11102...,https://nhadat24h.net.vn/du-an/ban-nha-pho-thu...,Có,Không,Không


In [ ]:
dataset.to_csv("/content/drive/MyDrive/newcode/all_dataset_after_schema-matching.csv", index=False, header=True)